In [1]:
import os
from glob import glob
import random
import time
import copy
import datetime as dt
import warnings
from collections import Counter
import itertools
import shutil
from pprint import pprint
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
import albumentations as A
import albumentations.pytorch
import sklearn
from sklearn.metrics import confusion_matrix
from joblib import parallel_backend
import ttach as tta
import timm
from timm.models.layers import Conv2dSame
from sklearn.metrics import f1_score
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.autograd import Variable
import torchvision.models as models
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정
time_now = dt.datetime.now()
run_id = time_now.strftime("%Y%m%d%H%M%S")

In [5]:
all_img_list = glob.glob('./train/*/*')

In [6]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[1])

In [7]:
df_2 = df[df['label'] == '2']
df_9 = df[df['label'] == '9']
df_10 = df[df['label'] == '10']
df_selected = pd.concat([df_2, df_9, df_10])

In [8]:
df_selected['label'].value_counts()

10    595
2     145
9      57
Name: label, dtype: int64

In [9]:
train, val, _, _ = train_test_split(df_selected, df_selected['label'], test_size=0.3, stratify=df_selected['label'], random_state=CFG['SEED'])

In [10]:
le = sklearn.preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [11]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [12]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(),
                            A.RandomContrast(limit=0.2),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [13]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)

train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

validation_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
validation_loader = DataLoader(validation_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

  0%|          | 0/557 [00:00<?, ?it/s]

  0%|          | 0/240 [00:00<?, ?it/s]

In [14]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_v2_l(pretrained=True)
        # self.backbone = timm.create_model('densenet201', pretrained=True, num_classes = 1000)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [15]:
def plot_confusion_matrix(
                        cm, classes, runid, epoch, 
                        f1, normalize=False, 
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(f'{title}-{runid}-{epoch}-{f1:.4f}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(f'./cms/cm-{runid}.jpg', dpi=400)
    plt.clf()

In [ ]:
test = pd.read_csv('./test.csv')
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

dataloaders = {
    'train': train_loader,
    'val': validation_loader,
    'test': test_loader
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(validation_dataset),
    'test': len(test_dataset)
}
fold = 0

# timm에서 모델을 가져옴
device =  torch.device("cuda")
# model = timm.create_model('tf_efficientnet_b7_ns', pretrained=True, num_classes=19)
model = BaseModel()
model.to(device)

epochs = 10  # 보통 30~40 epoch에서 멈춥니다.
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

os.makedirs(f'./runs/{run_id}', exist_ok=True)
os.makedirs(f'./cms/', exist_ok=True)

since = time.time()
best_f1 = 0.0
scaler = torch.cuda.amp.GradScaler()

fold_run_id = f'{run_id}_fold{str(fold)}'

# 학습
for epoch in range(epochs):
    print('-'*50)
    print(f'Fold: {fold}')
    print('Epoch {}/{}'.format(epoch, epochs - 1))
    train_loss = 0.0

    for phase in ['train', 'val']:
        running_loss = 0.0
        cm_preds = []
        cm_labels = []
        model_preds = []
        model_labels = []

        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        for x, y in tqdm(iter(dataloaders[phase])):
            x = x.to(device)
            y = y.type(torch.LongTensor).to(device)
            optimizer.zero_grad()

            r, flag= np.random.rand(1), 0
            
            # if r < 0.3: # CutMix
            # inputs, targets_a, targets_b, lam = cutmix_data(x, y) 
            # inputs, targets_a, targets_b = map(Variable, (inputs,
            #                                         targets_a, targets_b))
            
            # elif r > 0.6: # MixUp
                # inputs, targets_a, targets_b, lam = mixup_data(x, y)
                # inputs, targets_a, targets_b = map(Variable, (inputs,
                #                                         targets_a, targets_b))
            
            # else: flag = 0 # Original
            
            # inputs, targets_a, targets_b, lam = mixup_data(x, y)
            # inputs, targets_a, targets_b = map(Variable, (inputs,
            #                                             targets_a, targets_b))
            with torch.set_grad_enabled(phase == 'train'):
                with torch.cuda.amp.autocast(enabled=True):
                    y_hat = model(x)
                    if flag == 0: loss = criterion(y_hat, y)
                    # else: loss = mixup_criterion(criterion, y_hat, targets_a, targets_b, lam)
                _, preds = torch.max(y_hat, 1)

                if phase == 'train':
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

            running_loss += loss.item() * x.size(0)
            
            model_labels += y.detach().cpu().numpy().tolist()
            model_preds += preds.detach().cpu().numpy().tolist()

        if phase == 'train' and scheduler != None:
            scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_f1 = f1_score(
                    model_labels, 
                    model_preds, 
                    average='weighted')
        print(f'[{phase}] Loss: {epoch_loss:.4f} Weighted F1: {epoch_f1:.4f}')

        # 체크포인트 저장
        if phase == 'val':
            if epoch_f1 > best_f1:
                best_f1 = epoch_f1
                torch.save(model, f'./runs/{run_id}/best_model_fold{fold}.pt')
                confusion_mtx = confusion_matrix(model_labels, model_preds)
                plot_confusion_matrix(confusion_mtx, classes=le.classes_, runid=fold_run_id, epoch=epoch, f1=best_f1)
            else:
                # torch.save(model, f'./runs/{run_id}/{epoch}-val_loss{epoch_loss}-val_f1{epoch_f1}.pt')
                pass

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val_F1: {:4f}'.format(best_f1))

# 해당 fold의 checkpoint를 불러와 test
device =  torch.device("cuda")
checkpoint = f'./runs/{run_id}/best_model_fold{fold}.pt'
print(f'CHECKPOINT LOADED: {checkpoint}')
model = torch.load(checkpoint)
model.to(device)
model.eval()

test_preds = []

with torch.no_grad():
    for x in tqdm(iter(dataloaders['test'])):
            x = x.to(device)
            batch_pred = model(x)
            _, pred = torch.max(batch_pred, 1)
            pred = pred.detach().cpu().numpy().tolist()
            test_preds.extend(pred)

# trainset에 fit_trainsform 되어있는 LabelEncoder로 inverse transform 해줌
test_preds = le.inverse_transform(test_preds)

submit = pd.read_csv('./sample_submission.csv')
submit['label'] = test_preds

submit.loc[submit['label'] == '0', 'label'] = '가구수정'
submit.loc[submit['label'] == '1', 'label'] = '걸레받이수정'
submit.loc[submit['label'] == '2', 'label'] = '곰팡이'
submit.loc[submit['label'] == '3', 'label'] = '꼬임'
submit.loc[submit['label'] == '4', 'label'] = '녹오염'
submit.loc[submit['label'] == '5', 'label'] = '들뜸'
submit.loc[submit['label'] == '6', 'label'] = '면불량'
submit.loc[submit['label'] == '7', 'label'] = '몰딩수정'
submit.loc[submit['label'] == '8', 'label'] = '반점'
submit.loc[submit['label'] == '9', 'label'] = '석고수정'
submit.loc[submit['label'] == '10', 'label'] = '오염'
submit.loc[submit['label'] == '11', 'label'] = '오타공'
submit.loc[submit['label'] == '12', 'label'] = '울음'
submit.loc[submit['label'] == '13', 'label'] = '이음부불량'
submit.loc[submit['label'] == '14', 'label'] = '창틀,문틀수정'
submit.loc[submit['label'] == '15', 'label'] = '터짐'
submit.loc[submit['label'] == '16', 'label'] = '틈새과다'
submit.loc[submit['label'] == '17', 'label'] = '피스'
submit.loc[submit['label'] == '18', 'label'] = '훼손'

os.makedirs('./stack/', exist_ok=True)
submit.to_csv(f'./stack/Stacked_3_Classes.csv', index=False)
